In [ ]:
# LangGraph Agentic Workflow for Supply Chain with Chat History + Tavily + MongoDB + LLM Evaluation

from typing_extensions import TypedDict, NotRequired
from langgraph.graph import StateGraph, START, END
from langchain.tools import tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.tavily_search import TavilySearchResults
from pymongo import MongoClient
from datetime import datetime

# 1. Shared State
class CreateChainState(TypedDict):
    session_id: str
    raw_input: dict
    requirements: dict
    esg_preference: NotRequired[str]
    suppliers: list
    exploration_results: list
    evaluation_feedback: str
    final_report: dict

# 2. MongoDB Setup
MONGO_URI = "mongodb+srv://<username>:<password>@<cluster>.mongodb.net/?retryWrites=true&w=majority"
client = MongoClient(MONGO_URI)
db = client["supply_chain"]
suppliers_collection = db["suppliers"]
conversation_collection = db["chat_history"]
report_collection = db["final_reports"]

# 3. Log chat history

def log_message(role: str, content: str, session_id: str):
    conversation_collection.insert_one({
        "session_id": session_id,
        "role": role,
        "content": content,
        "timestamp": datetime.utcnow()
    })

# 4. Requirement Analysis Agent

def parse_requirements(state: CreateChainState):
    session_id = state["session_id"]
    user_input = state["raw_input"]
    log_message("user", str(user_input), session_id)

    search_terms = " ".join(str(v) for v in user_input.values() if isinstance(v, str))
    history_cursor = conversation_collection.find({"$text": {"$search": search_terms}, "role": "user"}).sort("timestamp", -1).limit(5)
    combined_requirements = dict(user_input)

    for msg in history_cursor:
        try:
            past_req = eval(msg["content"])
            if isinstance(past_req, dict):
                combined_requirements.update(past_req)
        except:
            pass

    return {"requirements": combined_requirements}

# 5. MongoDB Tool
@tool
def query_mongodb(requirements: dict, esg_filter: str = None) -> list:
    query = dict(requirements)
    if esg_filter:
        query["esg_score"] = esg_filter
    return list(suppliers_collection.find(query, {"_id": 0}))

# 6. Tavily Web Search Tool
@tool
def tavily_search_tool(requirements: dict) -> list:
    query = f"{requirements.get('material', '')} suppliers in {requirements.get('region', '')}"
    tavily = TavilySearchResults()
    results = tavily.run({"query": query})
    return [{"source": "tavily", "summary": r.get("content", r.get("snippet", "")), "url": r.get("url", "")} for r in results]

# 7. Supply Exploration Agent
def supply_exploration_agent(state: CreateChainState):
    mongo_results = query_mongodb(state["requirements"], state.get("esg_preference"))
    tavily_results = tavily_search_tool(state["requirements"])
    suppliers = mongo_results + tavily_results
    return {"suppliers": suppliers, "exploration_results": suppliers}

# 8. SupplyChainSageAgent using LLM
def supply_chain_sage_agent(state: CreateChainState):
    llm = ChatOpenAI(temperature=0, model_name="gpt-4")

    suppliers = state.get("suppliers", [])
    requirements = state.get("requirements", {})

    prompt_template = ChatPromptTemplate.from_template("""
    You are a world-class supply chain analyst. Analyze the following list of suppliers based on:
    - ESG score (Environmental, Social, Governance)
    - Delivery time (compared to required)
    - Pricing (if available)
    - Supplier reputation and credibility
    - Region matching
    - Certifications or compliance

    Requirements: {requirements}

    Suppliers:
    {suppliers}

    Return the top 5 suppliers in JSON format with fields:
    - name
    - score (1-10)
    - reasons (list)
    - warnings (list)
    - all available metadata from the supplier

    Conclude with either 'evaluation_feedback': 'good' or 'not good enough'
    """)

    messages = prompt_template.format_messages(
        requirements=requirements,
        suppliers=suppliers
    )
    response = llm(messages)

    from ast import literal_eval
    try:
        parsed = literal_eval(response.content)
        suppliers_filtered = parsed.get("top_suppliers", [])
        feedback = parsed.get("evaluation_feedback", "not good enough")
        return {"evaluation_feedback": feedback, "suppliers": suppliers_filtered}
    except:
        return {"evaluation_feedback": "not good enough", "suppliers": []}

# 9. Final Report Agent
def generate_final_report(state: CreateChainState):
    session_id = state["session_id"]
    if state["evaluation_feedback"] == "good":
        report = {
            "status": "success",
            "summary": f"{len(state['suppliers'])} top suppliers selected.",
            "top_suppliers": state["suppliers"],
            "evaluation_model": "gpt-4",
            "evaluation_notes": "Top 5 chosen based on score, reasoning, warnings"
        }
        report_collection.insert_one({"session_id": session_id, "report": report, "timestamp": datetime.utcnow()})
        log_message("assistant", str(report), session_id)
        return {"final_report": report}
    else:
        retry_msg = "Supply chain not good enough. Reanalyzing."
        log_message("assistant", retry_msg, session_id)
        return {"final_report": {"status": "retry", "message": retry_msg}}

# 10. Build LangGraph
builder = StateGraph(CreateChainState)
builder.add_node("parse", parse_requirements)
builder.add_node("explore", supply_exploration_agent)
builder.add_node("evaluate", supply_chain_sage_agent)
builder.add_node("report", generate_final_report)

builder.add_edge(START, "parse")
builder.add_edge("parse", "explore")
builder.add_edge("explore", "evaluate")

builder.add_conditional_edges("evaluate", lambda s: "retry" if s["evaluation_feedback"] == "not good enough" else "final", {
    "retry": "parse",
    "final": "report"
})
builder.add_edge("report", END)

create_chain_agent = builder.compile()

# 11. Sample Run
if __name__ == "__main__":
    user_input_dict = {
        "material": "copper",
        "region": "Asia",
        "delivery_time_days": {"$lte": 15}
    }

    initial_state = CreateChainState(
        session_id="session_001",
        raw_input=user_input_dict,
        esg_preference="high",
        requirements={}, suppliers=[], exploration_results=[], evaluation_feedback="", final_report={}
    )

    result = create_chain_agent.invoke(initial_state)
    print("\nFinal Output:")
    print(result["final_report"])

# 12. Sample MongoDB supplier seed data (run separately to populate the DB)
def seed_sample_suppliers():
    sample_suppliers = [
        {
            "name": "Asia Metals Ltd",
            "material": "copper",
            "region": "Asia",
            "delivery_time_days": 10,
            "esg_score": "high",
            "price": 150,
            "certifications": ["ISO 9001"],
            "reputation_score": 4
        },
        {
            "name": "Green Copper Co",
            "material": "copper",
            "region": "Asia",
            "delivery_time_days": 12,
            "esg_score": "medium",
            "price": 145,
            "certifications": ["ISO 14001"],
            "reputation_score": 3
        },
        {
            "name": "EcoMinerals",
            "material": "copper",
            "region": "Asia",
            "delivery_time_days": 20,
            "esg_score": "low",
            "price": 130,
            "reputation_score": 2
        },
        {
            "name": "Trusted Copper Supplies",
            "material": "copper",
            "region": "Asia",
            "delivery_time_days": 9,
            "esg_score": "high",
            "price": 160,
            "certifications": ["RoHS"],
            "reputation_score": 5
        },
        {
            "name": "Budget Copper Traders",
            "material": "copper",
            "region": "Asia",
            "delivery_time_days": 15,
            "esg_score": "medium",
            "price": 120,
            "reputation_score": 3
        }
    ]
    suppliers_collection.insert_many(sample_suppliers)

# To seed the database, uncomment and run:
# seed_sample_suppliers()


ModuleNotFoundError: No module named 'langgraph'

In [1]:
print("hello world")

hello world
